### Segmenting and Clustering Neighborhoods in Toronto


In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import requests
from pandas.io.json import json_normalize


In [4]:
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

import folium

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  26.50 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  10.60 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.08 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.53 MB/s
vincent-0.4.4- 100% |###################

In [2]:
from bs4 import BeautifulSoup

wiki="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_raw=requests.get(wiki).text

#BeautifulSoup to parse the code
soup_wiki = BeautifulSoup(wiki_raw,'xml')



In [3]:
table = soup_wiki.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

# extracting the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


In [10]:
#finding out unique postal codes

unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_unique      = []
Borough_unique       = []
Neighbourhood_unique = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_unique.append(p_var)
    Borough_unique.append(b_var)
    Neighbourhood_unique.append(n_var)

num of unique Postal codes: 77


In [11]:
#creating the dataframe
toronto = {'Postcode':Postcode_unique, 'Borough':Borough_unique, 'Neighbourhood':Neighbourhood_unique}
df_toronto = pd.DataFrame.from_dict(toronto)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Borough,Neighbourhood,Postcode
0,East York,Thorncliffe Park,M4H
1,North York,Parkwoods,M3A
2,Scarborough,Upper Rouge,M1X
3,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C
4,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",M5K
5,Etobicoke,"Alderwood, Long Branch",M8W
6,Downtown Toronto,Commerce Court,M5L
7,Scarborough,"Rouge, Malvern",M1B
8,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T
9,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L


In [ ]:
#rows and cols
df_toronto.shape
